In [1]:
!pip install transformers datasets


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 79.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 51.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 106.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 75.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 26.7 MB/s eta 0:00:00


In [6]:
!pip install sentencepiece


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 26.6 MB/s eta 0:00:00


In [1]:
from transformers import MarianMTModel, MarianTokenizer, Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq
from datasets import load_dataset, Dataset
import numpy as np
import glob
import json

In [27]:
# List all the files with the pattern "ptbr_phrases_*.json"
file_list = glob.glob("/content/ptbr_phrases_*.json")

# Load the contents of all the files into a single list
translated_phrases = []
for filename in file_list:
    with open(filename, 'r') as f:
        file_contents = json.load(f)
        translated_phrases.extend(file_contents)


In [28]:
translated_phrases[:10]

[['o capital da companhia devo ser desculpado:',
  "the company's capital must be excused:"],
 ['No primeiro eseru tinio ficarão sem maioria absoluta Caetano Furquim de Almeida e João Baptista da Fonseca, entre os quaes correu o segundo escrutinio.',
  'In the first ERA, Caetano Furquim de Almeida and João Baptista da Fonseca will be left without an absolute majority.'],
 ['O espirito da direcção.', 'The spirit of direction.'],
 ['B.', 'B.'],
 ['() Das circumstancias expostas resultou que, installada a Directoria, todas as esperanças se concentrarão em mim, que assim assumi grande responsabilidade.',
  '() From the circumstances set out above it has been found that, when the Directory is installed, all hopes will be concentrated on me, which I have thus assumed great responsibility.'],
 ['OTTONI 105 governo e cinco eleitos pelos accionistas;',
  'OTTONI 105 government and five elected by shareholders;'],
 ['Já disse no 2 que a experiencia me foi mostrando os defeitos com que foi organi

In [29]:
# Load parallel corpora
source_sentences = []  # List of source language sentences
target_sentences = []  # List of target language sentences

for i in translated_phrases:
  source_sentences.append(i[1])
  target_sentences.append(i[0])

# Combine source and target sentences
data = {"source": source_sentences, "target": target_sentences}
dataset = Dataset.from_dict(data)

# Split the dataset into train and validation sets
dataset = dataset.train_test_split(test_size=0.1)
train_dataset = dataset['train']
val_dataset = dataset['test']


In [5]:
model_name = "Helsinki-NLP/opus-mt-en-ROMANCE"  

tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

/usr/local/lib/python3.9/dist-packages/transformers/models/marian/tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [30]:
def tokenize_function(example):
    return tokenizer.prepare_seq2seq_batch(
        src_texts=example["source"],
        tgt_texts=example["target"],
        src_lang="en",
        tgt_lang="pt_br",  
        max_length=128,
        max_target_length=128,
        padding="max_length",
        truncation=True,
        return_tensors="pt"
    )

train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/77825 [00:00<?, ? examples/s]

/usr/local/lib/python3.9/dist-packages/transformers/tokenization_utils_base.py:3722: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and targets.

Here is a short example:

model_inputs = tokenizer(src_texts, text_target=tgt_texts, ...)

If you either need to use different keyword arguments for the source and target texts, you should do two calls like
this:

model_inputs = tokenizer(src_texts, ...)
labels = tokenizer(text_target=tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)
/usr/local/lib/python3.9/dist-packages/transformers/tokenization_utils_base.py:3596: UserWarning: `as_target_tokenizer` is deprec

Map:   0%|          | 0/8648 [00:00<?, ? examples/s]

In [31]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./output",
    evaluation_strategy="epoch",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    save_strategy="epoch",
    save_total_limit=2,
    learning_rate=5e-5,
    fp16=True,
    fp16_opt_level="O1",
    gradient_accumulation_steps=4,
    prediction_loss_only=True,
)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)


In [32]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer
)

trainer.train()


/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss
0,0.227700,0.191963
1,0.163800,0.164018
2,0.131400,0.153569


TrainOutput(global_step=14592, training_loss=0.1867814342442312, metrics={'train_runtime': 3894.6985, 'train_samples_per_second': 59.947, 'train_steps_per_second': 3.747, 'total_flos': 7914115666280448.0, 'train_loss': 0.1867814342442312, 'epoch': 3.0})

In [33]:
trainer.save_model("./output/ptbr_model")


In [34]:
def translate(text, trainer, max_length=128):
    device = trainer.model.device
    inputs = trainer.tokenizer.encode(text, return_tensors="pt", truncation=True, max_length=max_length).to(device)
    outputs = trainer.model.generate(inputs, max_length=max_length, num_return_sequences=1, no_repeat_ngram_size=2)

    decoded_output = trainer.tokenizer.decode(outputs[0], skip_special_tokens=True)
    return decoded_output


In [13]:
# Install the BLEURT package
!pip install --upgrade git+https://github.com/google-research/bleurt.git


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/google-research/bleurt.git to /tmp/pip-req-build-ytcam5ca
  Running command git clone --filter=blob:none --quiet https://github.com/google-research/bleurt.git /tmp/pip-req-build-ytcam5ca
  Resolved https://github.com/google-research/bleurt.git to commit cebe7e6f996b40910cfaa520a63db47807e3bf5c
  Preparing metadata (setup.py) ... done
  Created wheel for BLEURT: filename=BLEURT-0.0.2-py3-none-any.whl size=16456781 sha256=39ba13c77c81f0de9c4b557265936bcd2cf4f4bd3b7512d4ec641b335fb2cb67
  Stored in directory: /tmp/pip-ephem-wheel-cache-o0z1an00/wheels/b0/2a/c4/2bd63eb0e30d711baac17dfc89ca58a876cac68b44a2c2a97a
Successfully built BLEURT


In [14]:
!wget https://storage.googleapis.com/bleurt-oss-21/BLEURT-20.zip
!unzip BLEURT-20.zip


--2023-04-17 03:58:52--  https://storage.googleapis.com/bleurt-oss-21/BLEURT-20.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.153.128, 142.250.145.128, 74.125.128.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.153.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2140294207 (2.0G) [application/octet-stream]
Saving to: ‘BLEURT-20.zip’

BLEURT-20.zip       100%[===================>]   1.99G  30.0MB/s    in 68s     

2023-04-17 04:00:01 (29.9 MB/s) - ‘BLEURT-20.zip’ saved [2140294207/2140294207]

Archive:  BLEURT-20.zip
   creating: BLEURT-20/
  inflating: BLEURT-20/bert_config.json  
  inflating: BLEURT-20/saved_model.pb  
   creating: BLEURT-20/variables/
  inflating: BLEURT-20/variables/variables.index  
  inflating: BLEURT-20/variables/variables.data-00000-of-00001  
  inflating: BLEURT-20/sent_piece.vocab  
  inflating: BLEURT-20/bleurt_config.json  
  inflating: BLEURT-20/sent_piece.model  


In [17]:
!pip install openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.3/70.3 kB 6.5 MB/s eta 0:00:00


In [21]:
with open('key.txt', 'r') as file:
    OPENAI_API_KEY = file.read()

import openai

openai.api_key = OPENAI_API_KEY

prompt = "Can you translate the following into Brazilian Portuguese from the decade 1900 - 1909? For example anos would have been annos, ele would be elle, and traduçao would be traducçao. Here is the phrase, only return the translation: "


em pequeno cartão caprichosamente ilustrado, com capa colorida.


In [36]:

# Import the necessary libraries
from nltk.translate.bleu_score import sentence_bleu
from bleurt import score

# Load the BLEURT checkpoint
checkpoint = "BLEURT-20"
bleurt_scorer = score.BleurtScorer(checkpoint)

num_samples = len(translated_phrases[:100])
chatgpt_total_bleu_score = 0.0
total_bleu_score = 0.0
chatgpt_total_bleurt_score = 0.0
total_bleurt_score = 0.0

import random

# generate a list of 100 numbers randomly selected from 1 to 50000
random_list = random.sample(range(1, 50001), 100)

for i in random_list:    
    print(i)
    input_seq = translated_phrases[i:i+1][0][1]
    target_sentence = translated_phrases[i:i+1][0][0]
    response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are a translator from English to Portuguese."},
        {"role": "user", "content": prompt + input_seq}
    ]
    )
    chatgpt_sentence = response['choices'][0]['message']['content']
    translated_sentence = translate(input_seq, trainer)
    
    print(f"Source: {input_seq}")
    print(f"Target: {target_sentence}")
    print(f"ChatGPT 3.5: {chatgpt_sentence}")
    print(f"Fine-Tuned: {translated_sentence}")

    chatgpt_bleu_score = sentence_bleu([target_sentence.split()], chatgpt_sentence.split())
    chatgpt_total_bleu_score += chatgpt_bleu_score
    print(f"ChatGPT BLEU Score: {chatgpt_bleu_score}")   

    bleu_score = sentence_bleu([target_sentence.split()], translated_sentence.split())
    total_bleu_score += bleu_score
    print(f"Fine-Tuned BLEU Score: {bleu_score}")

    # Compute the BLEURT score
    chatgpt_bleurt_scores = bleurt_scorer.score(references=[target_sentence], candidates=[chatgpt_sentence])
    chatgpt_bleurt_score = chatgpt_bleurt_scores[0]
    chatgpt_total_bleurt_score += chatgpt_bleurt_score
    print(f"ChatGPT BLEURT Score: {chatgpt_bleurt_score}")

    bleurt_scores = bleurt_scorer.score(references=[target_sentence], candidates=[translated_sentence])
    bleurt_score = bleurt_scores[0]
    total_bleurt_score += bleurt_score
    print(f"Fine-Tuned BLEURT Score: {bleurt_score}")

    print("\n")

chatgpt_average_bleu_score = chatgpt_total_bleu_score / num_samples
print(f"Average ChatGPT BLEU Score: {chatgpt_average_bleu_score}")

chatgpt_average_bleurt_score = chatgpt_total_bleurt_score / num_samples
print(f"Average ChatGPT BLEURT Score: {chatgpt_average_bleurt_score}")


average_bleu_score = total_bleu_score / num_samples
print(f"Average BLEU Score: {average_bleu_score}")

average_bleurt_score = total_bleurt_score / num_samples
print(f"Average BLEURT Score: {average_bleurt_score}")


25789
Source: of the Poonite Seard intended for the assembly of all;
Target: do Seard pônite destinava a junção de todos;
ChatGPT 3.5: da Poonite Seard destinada a assembleia de todos.
Fine-Tuned: do Seard pônite destinado à assembléa de todos;
ChatGPT BLEU Score: 1.4256605770826504e-231
Fine-Tuned BLEU Score: 5.614021910443866e-78


/usr/local/lib/python3.9/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.9/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.9/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg

ChatGPT BLEURT Score: 0.7065920829772949
Fine-Tuned BLEURT Score: 0.6637528538703918


37878
Source: LI 116037 to 10380 Vol, 22.
Target: LI 116037 a 10380 Vol, 22.
ChatGPT 3.5: LI 116037 para 10380 Vol, 22. (No changes needed for this sentence as Brazilian Portuguese spelling rules did not change in the early 20th century.)
Fine-Tuned: LI 116037 a 10380 Vol, 22.
ChatGPT BLEU Score: 2.217630321330843e-78
Fine-Tuned BLEU Score: 1.0
ChatGPT BLEURT Score: 0.48899543285369873
Fine-Tuned BLEURT Score: 1.000630259513855


30123
Source: 000, nominal capital, 5th, Fundsings bonds, especially guaranteed by Alfandega's income.
Target: 000, capital nominal, 5 o, Fundings bonds, especialmente garantidos pelas rendas da Alfandega.
ChatGPT 3.5: 000, capital nominal, 5ª, Fundos de obrigações, especialmente garantidos pela receita da Alfândega.
Fine-Tuned: 000, capital nominal, 5o, Fundings bonds, especialmente garantidos pela renda da Alfandega.
ChatGPT BLEU Score: 3.908633169762327e-78
Fine-Tuned BLE